In [1]:
from rich.console import Console
from rich.columns import Columns
from rich.panel import Panel

console = Console()

items = [
    Panel("Item A"),
    Panel("Item B"),
    Panel("Item C"),
]

console.print(Columns(items))


╭────────╮ ╭────────╮ ╭────────╮
│ Item A │ │ Item B │ │ Item C │
╰────────╯ ╰────────╯ ╰────────╯

In [2]:
from rich.console import Console, Group
from rich.panel import Panel

console = Console()

body = Group(
    "[bold]Step 1[/bold]: Load data",
    "[bold]Step 2[/bold]: Process",
    "[bold]Step 3[/bold]: Save",
)

console.print(Panel(body, title="Pipeline"))

╭─────────────────────────────────────────────────── Pipeline ────────────────────────────────────────────────────╮
│ Step 1: Load data                                                                                               │
│ Step 2: Process                                                                                                 │
│ Step 3: Save                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [3]:
from rich.console import Console, Group
from rich.panel import Panel
from rich.text import Text

console = Console()

section = Group(
    Text("Header", style="bold magenta"),
    Panel("Body 1"),
    Panel("Body 2"),
)

console.print(section)

Header
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Body 1                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Body 2                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯